You are given a game scenario with classes Goblin and GoblinKing. Please implement the following rules:

    A goblin has base 1 attack/1 defense (1/1), a goblin king is 3/3.

    When the Goblin King is in play, every other goblin gets +1 Attack.

    Goblins get +1 to Defense for every other Goblin in play (a GoblinKing is a Goblin!).

Example:

    Suppose you have 3 ordinary goblins in play. Each one is a 1/3 (1/1 + 0/2 defense bonus).

    A goblin king comes into play. Now every goblin is a 2/4 (1/1 + 0/3 defense bonus from each other + 1/0 from goblin king)

The state of all the goblins has to be consistent as goblins are added and removed from the game.

Here is an example of the kind of test that will be run on the system:

    class FirstTestSuite(unittest.TestCase):
        def test(self):
            game = Game()
            goblin = Goblin(game)
            game.creatures.append(goblin)
     
            self.assertEqual(1, goblin.attack)
            self.assertEqual(1, goblin.defense)

Note: creature removal (unsubscription) does not need to be implemented.

In [27]:
from abc import ABC
from typing import Any
from enum import Enum

class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2

class Query:
    def __init__(self, initial_value: int, what_to_query: WhatToQuery):
        self.value = initial_value
        self.what_to_query = what_to_query

class Creature(ABC):
    def __init__(self, game: "Game", attack: int, defense: int):
        self.game = game
        self.base_attack = attack
        self.base_defense = defense

    def query(sender: "Creature", query: Query):
        pass

    def __str__(self) -> str:
        return "{} is a ({}/{})".format(type(self).__name__, self.attack, self.defense)
    
class Goblin(Creature):
    def __init__(self, game, attack=1, defense=1):
        super().__init__(game, attack, defense)

    @property
    def attack(self):
        query = Query(self.base_attack, WhatToQuery.ATTACK)

        for creature in self.game.creatures:
            creature.query(self, query)

        return query.value

    @property
    def defense(self):
        query = Query(self.base_attack, WhatToQuery.DEFENSE)

        for creature in self.game.creatures:
            creature.query(self, query)

        return query.value
    
    def query(self, sender: Creature, query: Query):
        if isinstance(sender, Goblin) and self is not sender and query.what_to_query == WhatToQuery.DEFENSE:
            query.value += 1


class GoblinKing(Goblin):
    def __init__(self, game, attack=3, defense=3):
        super().__init__(game, attack, defense)

    def query(self, sender: Creature, query: Query):
        if isinstance(sender, Goblin) and self is not sender and WhatToQuery.ATTACK:
            query.value += 1
        else:
            super().query(sender, query)
    

class Game:
    def __init__(self):
        self.creatures = []

In [28]:
game = Game()
game.creatures.append(Goblin(game))
game.creatures.append(Goblin(game))
game.creatures.append(Goblin(game))
game.creatures.append(GoblinKing(game))

for c in game.creatures:
    print(c)

Goblin is a (2/4)
Goblin is a (2/4)
Goblin is a (2/4)
GoblinKing is a (3/6)
